In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
import sklearn
import sklearn.model_selection
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


In [2]:
Train_data = pd.read_csv('/kaggle/input/30-days-of-ml/train.csv').set_index("id")
X,y= Train_data.drop(['target'], axis=1)  ,   Train_data.target 
cols_with_missing = [col for col in X.columns if X[col].isnull().any()]
cols_with_letter_values = [{col:X[col].unique()} for col in X if X[col].dtypes == 'object' ]
object_cols = [col for col in X if X[col].dtypes == 'object' ]

In [3]:
cut_off=5

In [4]:
#orordinal_ encoder<<<<<<<<<<<<<<<<<<< 
ordinal_encoder = sklearn.preprocessing.OrdinalEncoder()
X[object_cols[cut_off:]] = ordinal_encoder.fit_transform(X[object_cols[cut_off:]])
print('ordinary endcoding on {}'.format(object_cols[cut_off:]))
#one hot encoding<<<<<<<<<<<<<< for 1-4 or maybe from 1-8
OH_encoder = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X[object_cols[:cut_off]])) # the cut off happens here ----------------
OH_cols_train.index = X.index
num_X_train = X.drop(object_cols[:cut_off], axis=1)
X = pd.concat([num_X_train, OH_cols_train], axis=1)
print('One-Hot endcoding on {}'.format(object_cols[:cut_off]))

ordinary endcoding on ['cat5', 'cat6', 'cat7', 'cat8', 'cat9']
One-Hot endcoding on ['cat0', 'cat1', 'cat2', 'cat3', 'cat4']


In [5]:
X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X, y)

In [6]:
#RandomForestRegressor
Forest_model=sklearn.ensemble.RandomForestRegressor()
Forest_model.fit(X_train,y_train)


RandomForestRegressor()

In [7]:
preds_valid = Forest_model.predict(X_valid)
print(mean_squared_error(y_valid, preds_valid, squared=False))

0.7344269037864378


In [8]:
test = pd.read_csv("../input/30-days-of-ml/test.csv", index_col=0)

In [9]:
ordinal_encoder = sklearn.preprocessing.OrdinalEncoder()
test[object_cols[cut_off:]] = ordinal_encoder.fit_transform(test[object_cols[cut_off:]])
print('ordinary endcoding on {}'.format(object_cols[cut_off:]))
#one hot encoding<<<<<<<<<<<<<< for 1-4 or maybe from 1-8
OH_encoder = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(test[object_cols[:cut_off]])) # the cut off happens here ----------------
OH_cols_train.index = test.index
num_X_train = test.drop(object_cols[:cut_off], axis=1)
test = pd.concat([num_X_train, OH_cols_train], axis=1)
print('One-Hot endcoding on {}'.format(object_cols[:cut_off]))

ordinary endcoding on ['cat5', 'cat6', 'cat7', 'cat8', 'cat9']
One-Hot endcoding on ['cat0', 'cat1', 'cat2', 'cat3', 'cat4']


In [10]:
predictions = Forest_model.predict(test)

In [11]:
output = pd.DataFrame({'Id': test.index,
                       'target': predictions})
output.to_csv('submission.csv', index=False)